# Load libraries

In [1]:
import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import pinecone
from langchain.vectorstores import Pinecone

import os
from dotenv import load_dotenv

/Users/kieranschubert/Desktop/unigpt-assist/venv_unigpt/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Load dotenv

In [2]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')
PINECONE_INDEX = os.getenv('PINECONE_INDEX')

# Init Pinecone VectorDB

In [3]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
)

if PINECONE_INDEX not in pinecone.list_indexes():
    pinecone.create_index(PINECONE_INDEX, dimension=1536, metric="cosine")

index = pinecone.Index(PINECONE_INDEX)

print(pinecone.list_indexes())
print(pinecone.describe_index(PINECONE_INDEX))
print(index.describe_index_stats())

['hackathon']
IndexDescription(name='hackathon', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')
{'dimension': 1536,
 'index_fullness': 0.00019,
 'namespaces': {'': {'vector_count': 19}},
 'total_vector_count': 19}


In [4]:
embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002",
                               disallowed_special=())

vectorstore = Pinecone.from_existing_index(
            index_name=PINECONE_INDEX,
            embedding=embedding_function,
            #namespace=namespace
            )

k = 10

retriever = vectorstore.as_retriever(search_kwargs={"k": k}, return_source_documents=True)

In [5]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model="gpt-4-1106-preview",
                 temperature=0)

In [6]:
template = """
        Vous êtes un assistant qui répond à des questions sur l'Université de Genève, basée en Suisse.
        Utilisez les éléments de contexte et l'historique du chat suivants pour répondre aux questions. 
        Votre réponse doit être liée à l'Université de Genève uniquement. Si la question ne figure pas dans le contexte ou l'historique du chat, répondez "Je suis désolé, je ne connais pas la réponse".
        Les réponses doivent être détaillées mais concises et courtes.
        Respirez profondément et travaillez étape par étape.

        Historique: {chat_history}
        
        Context: {context}

        Question: {question}
        Answer: """

prompt = PromptTemplate(input_variables=["chat_history", "context", "question"], template=template)

In [7]:
from langchain.chains.conversation.memory import ConversationBufferMemory

In [33]:
conversational_memory = ConversationBufferWindowMemory(
        memory_key='chat_history',
        input_key="question",
        k=3,
        return_messages=True
    )

In [34]:
qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt,
                               "memory": conversational_memory},
            return_source_documents=True,
            verbose=False,
            )

In [41]:
res = qa({"query": "bonjour?"})

In [42]:
conversational_memory.load_memory_variables({})

{'chat_history': [HumanMessage(content="J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?"),
  AIMessage(content='La durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire et 2 semestres pour une maîtrise universitaire.'),
  HumanMessage(content="J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?"),
  AIMessage(content='La durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire et 2 semestres pour une maîtrise universitaire.'),
  HumanMessage(content='bonjour?'),
  AIMessage(content="Bonjour! Comment puis-je vous aider concernant l'Université de Genève?")]}

In [32]:
res = qa({"query": "Je suis en troisième année de médecine puis-je prendre une année sabatique ?",
          "chat_history": mem})

In [11]:
res["query"]

'Je suis en troisième année de médecine puis-je prendre une année sabatique ?'

In [12]:
res["result"]

"Oui, en tant qu'étudiant à l'Université de Genève, vous pouvez demander un congé, qui peut être accordé pour une période d'un semestre ou d'une année et est renouvelable. Cependant, la durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire. Vous devez adresser votre demande écrite au doyen au minimum un mois avant le début du semestre pour lequel vous souhaitez obtenir un congé. Pendant le congé, vous ne pouvez ni assister aux cours ou aux travaux pratiques, ni vous présenter aux évaluations."

In [13]:
res["source_documents"]

[Document(page_content="6 – Congé et études à temps partiel  \n1. Le doyen peut accorder à l'étudiant qui en fait la demande écrite un congé. \n \n2. Le congé est accordé pour une période d’un semestre ou d’une année, il  est \nrenouvelable. Sauf exception, la durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire et 2 semestres pour une maîtrise universitaire. \n 3. Pendant l'interruption accordée (congé), l'étudiant ne peut ni assister aux cours  ou \naux travaux pratiques, ni se présenter aux évaluations. \n 4. Les demandes doivent être adressées au minimum un mois avant le début du \nsemestre pour lequel l’étudiant souhaite obtenir un congé. \n \n5. Des études à temps partiel peuvent être envisagées dans certains cas. Les \ndemandes dûment motivées doivent être adressées au doyen au minimum un mois avant le début de l’année académique pour laquelle l’étudiant souhaite obtenir un temps partiel. Le doyen ou, par délégation, le vice- doyen en charge des é

In [39]:
qa.combine_documents_chain.memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content="J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?"), AIMessage(content='La durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire et 2 semestres pour une maîtrise universitaire.'), HumanMessage(content="J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?"), AIMessage(content='La durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire et 2 semestres pour une maîtrise universitaire.'), HumanMessage(content="J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?"), AIMessage(content='La durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire et 2 semestres pour une maîtrise universitaire.'), HumanMessage(content="J'ai envie de prendre un congé durant mes études, quelle est la durée maximum autorisée?")

In [15]:
qa.combine_documents_chain.memory.chat_memory

ChatMessageHistory(messages=[HumanMessage(content='Je suis en troisième année de médecine puis-je prendre une année sabatique ?'), AIMessage(content="Oui, en tant qu'étudiant à l'Université de Genève, vous pouvez demander un congé, qui peut être accordé pour une période d'un semestre ou d'une année et est renouvelable. Cependant, la durée totale du congé ne peut excéder 3 semestres pour un baccalauréat universitaire. Vous devez adresser votre demande écrite au doyen au minimum un mois avant le début du semestre pour lequel vous souhaitez obtenir un congé. Pendant le congé, vous ne pouvez ni assister aux cours ou aux travaux pratiques, ni vous présenter aux évaluations.")])

In [30]:
qa.memory

In [23]:
dir(qa)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [19]:
from langchain.memory import ConversationBufferWindowMemory

In [20]:
memory = ConversationBufferWindowMemory( k=1)

In [21]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [22]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}